In [1]:
from pathlib import Path

import mlx.core as mx
import mlx.nn as nn
import utils
from mlx.utils import tree_flatten, tree_unflatten
from models import LoRALinear

/Users/mab/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = "Trendyol/Trendyol-LLM-7b-chat-v0.1"
save_path = "my-models"
adapter_file = "adapters/20240616-202312-adapters-all.npz"
de_quantize = False
upload_name = None
hf_path = None

In [3]:
model, tokenizer, config = utils.load(model)

Fetching 12 files: 100%|██████████| 12/12 [00:00<00:00, 95144.89it/s]


In [4]:
# Load adapters and get number of LoRA layers
adapters = list(mx.load(adapter_file).items())
lora_layers = len([m for m in adapters if "q_proj.lora_a" in m[0]])
lora_layers

16

In [5]:
# Freeze all layers other than LORA linears
model.freeze()

Model(
  (model): LlamaModel(
    (embed_tokens): Embedding(44222, 4096)
    (layers.0): TransformerBlock(
      (self_attn): Attention(
        (q_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (k_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (v_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (o_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (rope): RoPE(128, traditional=False)
      )
      (mlp): MLP(
        (gate_proj): Linear(input_dims=4096, output_dims=11008, bias=False)
        (down_proj): Linear(input_dims=11008, output_dims=4096, bias=False)
        (up_proj): Linear(input_dims=4096, output_dims=11008, bias=False)
      )
      (input_layernorm): RMSNorm(4096, eps=1e-05)
      (post_attention_layernorm): RMSNorm(4096, eps=1e-05)
    )
    (layers.1): TransformerBlock(
      (self_attn): Attention(
        (q_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (k_proj): L

In [6]:

for l in model.model.layers[len(model.model.layers) - lora_layers :]:
    l.self_attn.q_proj = LoRALinear.from_linear(l.self_attn.q_proj)
    l.self_attn.v_proj = LoRALinear.from_linear(l.self_attn.v_proj)
    if hasattr(l, "block_sparse_moe"):
        l.block_sparse_moe.gate = LoRALinear.from_linear(l.block_sparse_moe.gate)


In [7]:
model.update(tree_unflatten(adapters))
fused_linears = [
    (n, m.to_linear())
    for n, m in model.named_modules()
    if isinstance(m, LoRALinear)
]

In [8]:
model.update_modules(tree_unflatten(fused_linears))

Model(
  (model): LlamaModel(
    (embed_tokens): Embedding(44222, 4096)
    (layers.0): TransformerBlock(
      (self_attn): Attention(
        (q_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (k_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (v_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (o_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (rope): RoPE(128, traditional=False)
      )
      (mlp): MLP(
        (gate_proj): Linear(input_dims=4096, output_dims=11008, bias=False)
        (down_proj): Linear(input_dims=11008, output_dims=4096, bias=False)
        (up_proj): Linear(input_dims=4096, output_dims=11008, bias=False)
      )
      (input_layernorm): RMSNorm(4096, eps=1e-05)
      (post_attention_layernorm): RMSNorm(4096, eps=1e-05)
    )
    (layers.1): TransformerBlock(
      (self_attn): Attention(
        (q_proj): Linear(input_dims=4096, output_dims=4096, bias=False)
        (k_proj): L

In [9]:
if de_quantize:
    de_quantize_layers = []
    for n, m in model.named_modules():
        if isinstance(m, nn.QuantizedLinear):
            bias = "bias" in m
            weight = m.weight
            weight = mx.dequantize(
                weight,
                m.scales,
                m.biases,
                m.group_size,
                m.bits,
            ).astype(mx.float16)
            output_dims, input_dims = weight.shape
            linear = nn.Linear(input_dims, output_dims, bias=bias)
            linear.weight = weight
            if bias:
                linear.bias = m.bias
            de_quantize_layers.append((n, linear))

    model.update_modules(tree_unflatten(de_quantize_layers))

In [10]:
weights = dict(tree_flatten(model.parameters()))
if de_quantize:
    config.pop("quantization", None)
utils.save_model(save_path, weights, tokenizer, config)


In [11]:
if upload_name is not None:
    if not Path(model).exists():
        # If the model path doesn't exist, assume it's an HF repo
        hf_path = model
    elif hf_path is None:
        raise ValueError(
            "Must provide original Hugging Face repo to upload local model."
        )
    utils.upload_to_hub(save_path, upload_name, hf_path)

In [12]:
def generate(model, prompt, tokenizer, temp, max_tokens):
    generated_text = ""
    print(prompt, end="", flush=True)

    prompt = mx.array(tokenizer.encode(prompt))

    tokens = []
    skip = 0
    for token, n in zip(
        utils.generate(prompt, model, temp),
        range(max_tokens),
    ):
        if token == tokenizer.eos_token_id:
            break

        tokens.append(token.item())
        s = tokenizer.decode(tokens)
        if len(s) - skip > 1:
            print(s[skip:-1], end="", flush=True)
            skip = len(s) - 1
    print(tokenizer.decode(tokens)[skip:], flush=True)
    print("=" * 20)

    generated_text = tokenizer.decode(tokens)
    if len(tokens) == 0:
        print("No tokens generated for this prompt")
        return
    return generated_text

In [13]:
example_prompt = "Merhaba sol femur alt uç kırığı ve kalça kırığı çivisi takılmıştı fazlasıyla rahatsız ediyor aldiricam sonrasında ne zamana kadar normal yasantima dönerim?"

In [16]:
temp = 0.7
max_tokens = 200
generated_text = generate(model, example_prompt, tokenizer, temp, max_tokens)

Merhaba sol femur alt uç kırığı ve kalça kırığı çivisi takılmıştı fazlasıyla rahatsız ediyor aldiricam sonrasında ne zamana kadar normal yasantima dönerim?Teşekkür ederim Sayın Prof. Dr. Aybars AKKOR Aybars Bey, Kırık kırık. Eklem cerrahisi ile tedavi edilmesi gereken bir durum. Bu tedavinin değişmeyen bir şartı var, o da fizik tedavi. Fizik tedavi doktorunuz hangi sıklıkla, hangi günlerde ve hangi sürelerle yapılacağını size söyleyecektir. Fizik tedavi süresince normal yasarlıkınıza dönüşmeniz mümkün olmaz. Bu nedenle şimdiden kendinizi doktorunuzun önerilerine göre hazırlamanızda fayda var. İyi günler. Dr Aybars Akkor *** *** ** / *** *** ** / *** ** / *** ** / ** - Doktorlarımızın birçoğu gelen sorulardan hazırlıksız cevap veriyor. Sorularınızı daha iyi yanıtlayabilmemiz için lütfen daha detaylı bilgi verin. *** *** ** / ** *** ** / ** *** ** / ** *** ** / **   Ankara Kırıkkale de yaşıyorum ve 2 yıldır


In [15]:
generated_text

"Teşekkür ederim..### Assistant: Merhaba,ameliyat sonrası iyileşme dönemi kişiden kişiye değişse de ortalama 3 ila 6 ay kadar sürüyor.###<unk> : Merhaba doktor hanım benim sol femur alt uc yakınımında kırılma var ve kalça civi takıldı sürekli bacakta ağri ve hareketsizlik var. Zamanla düzelme olucakmi ve bu süre ne kadar? Teşekkür ederim. Saygılarımla. Dr. Ali Akben: Merhaba, bacakta hareketsizlik ve ağrılar cerrahi müdahalelerden sonra olur. Ameliyatınızı kaç gün önce ve hangi cerrahınız yapmış? Geçmiş olsun, Dr. Ali Akben. Dr. Akben'e bu soruyu sordusunuz, cevap verdi. Dr. Akben: Merhaba. Ameliyatı ameliyatı yapan cerrahtan randevu alarak yapınız. Dr."